In [17]:
# Import dependencies
import pandas as pd

In [18]:
# Read csv file into a DataFrame
games = pd.read_csv("bgg_dataset.csv", delimiter=';')
games.head()

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,"8,61",2,"2,84",65294.0,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,70",5,"4,22",16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


In [19]:
# Checking for null values
games.isnull().sum()

ID                       16
Name                      0
Year Published            1
Min Players               0
Max Players               0
Play Time                 0
Min Age                   0
Users Rated               0
Rating Average            0
BGG Rank                  0
Complexity Average        0
Owned Users              23
Mechanics              1598
Domains               10159
dtype: int64

In [20]:
# Review the games with no ID
noID= games[games["ID"].isnull()]
noID

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
10776,NaN,Ace of Aces: Jet Eagles,1990.0,2,2,20,10,110,"6,26",10778,2,NaN,NaN,NaN
10835,NaN,Die Erben von Hoax,1999.0,3,8,45,12,137,"6,05",10837,2,NaN,NaN,NaN
11152,NaN,Rommel in North Africa: The War in the Desert ...,1986.0,2,2,0,12,53,"6,76",11154,4,NaN,NaN,NaN
11669,NaN,Migration: A Story of Generations,2012.0,2,4,30,12,49,"7,20",11671,"2,00",NaN,NaN,NaN
12649,NaN,Die Insel der steinernen Wachter,2009.0,2,4,120,12,49,"6,73",12651,3,NaN,NaN,NaN
12764,NaN,Dragon Ball Z TCG (2014 edition),2014.0,2,2,20,8,33,"7,03",12766,"2,50",NaN,NaN,NaN
13282,NaN,Dwarfest,2014.0,2,6,45,12,82,"6,13",13284,"1,75",NaN,NaN,NaN
13984,NaN,Hus,NaN,2,2,40,0,38,"6,28",13986,2,NaN,NaN,NaN
14053,NaN,Contrario 2,2006.0,2,12,0,14,37,"6,30",14055,"1,00",NaN,NaN,NaN
14663,NaN,Warage: Extended Edition,2017.0,2,6,90,10,49,"7,64",14665,3,NaN,NaN,NaN


In [21]:
# Drop rows that have more than 2 missing values
games.dropna(thresh=len(games.columns)-2, inplace =True)

In [22]:
games.isnull().sum()

ID                        0
Name                      0
Year Published            0
Min Players               0
Max Players               0
Play Time                 0
Min Age                   0
Users Rated               0
Rating Average            0
BGG Rank                  0
Complexity Average        0
Owned Users               6
Mechanics              1581
Domains               10142
dtype: int64

In [23]:
# Move mechanics column into it's own DataFrame. 
mechanics = games[['ID', 'Mechanics']]
mechanics.head()

,ID,Mechanics
0,174430.0,"Action Queue, Action Retrieval, Campaign / Bat..."
1,161936.0,"Action Points, Cooperative Game, Hand Manageme..."
2,224517.0,"Hand Management, Income, Loans, Market, Networ..."
3,167791.0,"Card Drafting, Drafting, End Game Bonuses, Han..."
4,233078.0,"Action Drafting, Area Majority / Influence, Ar..."


In [24]:
# Drop Mechanics field from games DF
games = games.drop('Mechanics', axis=1)
games.head()

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Strategy Games, Thematic Games"
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,"8,61",2,"2,84",65294.0,"Strategy Games, Thematic Games"
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,Strategy Games
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,Strategy Games
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,70",5,"4,22",16831.0,"Strategy Games, Thematic Games"


'9,58'